# Point Cloud Diffusion 

## Imports

In [ ]:
# Dataset-related
from pclab.datasets.modelnet import ModelNet40
from pclab.transforms import *
from torch.utils.data import DataLoader

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim

# Visualization
from pclab.utils import quick_vis

# Other
from functools import partial
from pclab.models.pointnet import LinearLayer

# Lightning
import pytorch_lightning as pl

## Dataset

In [ ]:
path = "/home/ioannis/Desktop/programming/data"

transforms = [RandomPointKeep(1024), UnitSphereNormalization(), AnisotropicScale(), ToTensor()]

train_dataset = ModelNet40(path, 2048, 'train', transforms=transforms, category=0)
valid_dataset = ModelNet40(path, 2048, 'test' , transforms=transforms, category=0)

train_dl, valid_dl = map(partial(DataLoader, batch_size=32, shuffle=True, num_workers=8, drop_last=True), (train_dataset, valid_dataset))
len(train_dl)

19

## Point AutoEncoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        
        self.ln1 = LinearLayer(

In [ ]:
?? LinearLayer

Init signature:
 LinearLayer(
    in_channels,
    out_channels,
    fc=False,
    use_norm=True,
    use_relu=True,
)
Source:        
class LinearLayer(nn.Module):
    "A linear layer that can be either used as a shared-mlp or as a block in a fully connected network"
    def __init__(self, in_channels, out_channels, fc=False, use_norm=True, use_relu=True):
        super().__init__()

        # we can skip the bias if we use batch normalization
        bias = not use_norm

        layers = [nn.Linear(in_channels, out_channels, bias=bias) if fc else nn.Conv1d(in_channels, out_channels, kernel_size=1, bias=bias)]

        if use_norm:
            layers.append(nn.BatchNorm1d(out_channels))

        if use_relu:
            layers.append(nn.ReLU())

        self.net = nn.Sequential(*layers)

        # initializing the weights of the module
        self.init_weights()

    def init_weights(self):
        init.kaiming_uniform_(self.net[0].weight, a=math.sqrt(2))
        # bias initializatio